In [30]:
import pandas as pd
import re 
import os

os.getcwd() # Print current working directory

'/Users/cjgimena/Desktop/Github/consulting-spring2025/season_report/return_page'

Read Data

In [31]:
os.chdir('../..') # go back two directories
os.getcwd()

'/Users/cjgimena/Desktop/Github/consulting-spring2025'

In [32]:
# USER INPUT: Specify Player!
player_name = 'Rudy Quan'
# player_name = 'Kaylan Bigun'
# # player_name = 'Emon Van Loben Sels'
# player_name = 'Alexander Hoogmartens'
# player_name = 'Spencer Johnson'
# player_name = 'Aadarsh Tripathi'
# player_name = 'Giacomo Revelli'
# player_name = 'Gianluca Ballotta'

In [33]:
combined_data_shots = pd.read_excel(f'data/mens/{player_name}/combined.xlsx', sheet_name='Shots')
combined_data_points = pd.read_excel(f'data/mens/{player_name}/combined.xlsx', sheet_name='Points')
combined_data_games = pd.read_excel(f'data/mens/{player_name}/combined.xlsx', sheet_name='Games')
combined_data_sets = pd.read_excel(f'data/mens/{player_name}/combined.xlsx', sheet_name='Sets')

In [34]:
# Subset 2024-2025 Season Matches!
mens_results = pd.read_csv('data/mens/mens_results.csv')[:229]

# Change Date Format
mens_results['Date'] = pd.to_datetime(mens_results['Date'])


def filter_player(data, player_name):

    # Filter for player_name
    data = data[(data['Player1'] == player_name) | (data['Player2'] == player_name)]

    # Filter for only school events
    data = data[data['Event Name'].str.startswith(('Dual Match', '2024 ITA', '2024-25 NCAA Division'))]
    return data

mens_results_player = filter_player(mens_results, player_name)

In [35]:
mens_results_player

,Event Name,Date,Player1,Player2,Player1 UTR,Player2 UTR,Score
5,"Dual Match: University of California, Los Ange...",2025-05-15,Timo Legout,Rudy Quan,14.23,13.59,"7-5, 6-2"
6,Dual Match: University of Southern California ...,2025-05-08,Rudy Quan,Makk Peter,13.59,13.79,"7-6(4), 3-6, 3-3"
14,"Dual Match: University of California, Los Ange...",2025-05-02,Carl Overbeck,Rudy Quan,13.48,13.59,"5-7, 7-5, 6-3"
18,"Dual Match: University of California, Santa Ba...",2025-05-01,Rudy Quan,Gianluca Brunkow,13.59,13.00,"5-7, 6-2, 0-1"
27,"Dual Match: University of California, Los Ange...",2025-04-26,Aidan Kim,Rudy Quan,13.74,13.59,"7-6(3), 6-2"
33,Dual Match: Michigan State University vs Unive...,2025-04-25,Rudy Quan,Aristotelis Thanos,13.59,13.67,"4-6, 3-1"
36,Dual Match: University of Michigan vs Universi...,2025-04-24,Rudy Quan,William Cooksey,13.59,12.00,"6-3, 5-6"
46,"Dual Match: University of California, Los Ange...",2025-04-19,Rudy Quan,Calvin MUELLER,13.59,13.00,"3-6, 6-3, 6-1"
52,"Dual Match: University of California, Los Ange...",2025-04-17,Michael Minasyan,Rudy Quan,12.00,13.59,"2-6, 6-3"
55,Dual Match: Michigan State University vs Unive...,2025-04-12,Rudy Quan,Ozan Baris,13.59,13.66,"6-1, 6-2"


Percent of return wins function

In [36]:
combined = combined_data_shots.merge(combined_data_points[['Point', 'Game', 'Set', 'Point Winner', 'Match Server', '__source_file__']], 
                          how = 'left', 
                          on = ['Set', 'Game', 'Point', '__source_file__']).head()

In [39]:
def return_wins_percent():
    #find all of the players return games
    returns = combined[(combined['Player'] == player_name) & (combined['Type'] == 'return')]
    #makes a new column for the point winner and determines whether host or guest was returning
    returns['ReturnerSide'] = returns['Match Server'].apply(
        lambda s: 'Guest' if s == 'Host' else 'Host'
    )
    returns['ReturnWon'] = returns['Point Winner'] == returns['ReturnerSide']

    #Compute percentage
    if len(returns) == 0:
        return 0.0
    pct = returns['ReturnWon'].mean() * 100
    return round(pct, 2)


### Cj Work

In [16]:
import pandas as pd
import re 
import os

player_name = 'Rudy Quan'

combined_data_shots = pd.read_excel(f'../../data/mens/{player_name}/combined.xlsx', sheet_name='Shots')
combined_data_points = pd.read_excel(f'../../data/mens/{player_name}/combined.xlsx', sheet_name='Points')
combined_data_games = pd.read_excel(f'../../data/mens/{player_name}/combined.xlsx', sheet_name='Games')
combined_data_sets = pd.read_excel(f'../../data/mens/{player_name}/combined.xlsx', sheet_name='Sets')
combined_data_settings = pd.read_excel(f'../../data/mens/{player_name}/combined.xlsx', sheet_name='Settings')

In [8]:
combined = combined_data_shots.merge(combined_data_points[['Point', 'Game', 'Set', 'Point Winner', 'Match Server', '__source_file__']], 
                          how = 'left', 
                          on = ['Set', 'Game', 'Point', '__source_file__'])[1000:4000]

combined['Type'].unique()

array(['first_serve', 'first_return', 'serve_plus_one', 'return_plus_one',
       'none', 'second_serve', 'second_return', 'in_play', 'serve',
       'return'], dtype=object)

In [28]:
# Find All Returning Games
returns = combined[(combined['Match Server'] == 'guest') & ((combined['Type'] == 'first_return') | (combined['Type'] == 'second_return'))]
returns

,Player,Shot,Type,Stroke,Spin,Speed (MPH),Point,Game,Set,Bounce Depth,...,Hit (y),Hit (z),Direction,Result,Favorited,Start Time,Video Time,__source_file__,Point Winner,Match Server
1009,Rudy Quan,2,second_return,Backhand,Topspin,51.582260,37,8,1,deep,...,24.587389,1.673195,inside out,In,False,21:37:04,1439.130005,RudyQuan_FloridaState_9_23_24.xlsx,host,guest
1016,Rudy Quan,2,second_return,Backhand,Flat,71.305695,38,8,1,deep,...,26.304169,1.149330,down the line,In,False,21:37:38,1473.880005,RudyQuan_FloridaState_9_23_24.xlsx,guest,guest
1026,Rudy Quan,2,first_return,Backhand,Topspin,40.496258,39,8,1,short,...,25.424723,1.853700,inside in,In,False,21:38:18,1513.339966,RudyQuan_FloridaState_9_23_24.xlsx,guest,guest
1028,Rudy Quan,2,first_return,Backhand,Topspin,50.528534,40,8,1,deep,...,25.525892,1.048352,down the line,In,False,21:38:40,1535.949951,RudyQuan_FloridaState_9_23_24.xlsx,guest,guest
1034,Rudy Quan,2,first_return,Backhand,Slice,32.772774,41,8,1,deep,...,25.731417,1.620931,inside out,In,False,21:39:07,1562.339966,RudyQuan_FloridaState_9_23_24.xlsx,guest,guest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3947,Rudy Quan,2,first_return,Forehand,Flat,53.472031,85,15,2,out,...,27.328449,1.033211,inside out,Out,False,22:48:06,NaN,RudyQuan_Pepperdine_2_27_25.xlsx,guest,guest
3951,Rudy Quan,2,first_return,Backhand,Flat,55.458935,86,15,2,deep,...,26.492403,1.157648,inside in,In,False,22:48:22,NaN,RudyQuan_Pepperdine_2_27_25.xlsx,guest,guest
3959,Rudy Quan,2,second_return,Forehand,Flat,77.942482,87,15,2,deep,...,25.763754,1.210788,inside out,In,False,22:49:06,NaN,RudyQuan_Pepperdine_2_27_25.xlsx,host,guest
3964,Rudy Quan,2,first_return,Forehand,Topspin,61.407204,88,15,2,deep,...,27.128525,1.075889,down the line,In,False,22:49:57,NaN,RudyQuan_Pepperdine_2_27_25.xlsx,guest,guest


In [41]:
combined_data_games[['Server', 'Game Winner']]

,Server,Game Winner
0,host,draw
1,host,guest
2,guest,host
3,host,host
4,guest,guest
...,...,...
147,host,guest
148,guest,guest
149,host,host
150,guest,guest


In [ ]:
def return_wins_percent(data):
    
    # find all of the players return games
    returns = combined[(combined['Player'] == player_name) & (combined['Type'] == 'return')]
    
    # makes a new column for the point winner and determines whether host or guest was returning
    returns['ReturnerSide'] = returns['Match Server'].apply(
        lambda s: 'Guest' if s == 'Host' else 'Host'
    )
    returns['ReturnWon'] = returns['Point Winner'] == returns['ReturnerSide']

    #Compute percentage
    if len(returns) == 0:
        return 0.0
    pct = returns['ReturnWon'].mean() * 100
    return round(pct, 2)

return_wins_percent(mens_results_player)
